<a href="https://colab.research.google.com/github/LSSTDESC/DifferentiableHOS/blob/main/notebooks/spectrum_for_cosmology.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pylab inline 
%load_ext autoreload
%autoreload 2
import tensorflow as tf
import numpy as np
import sys
sys.path.insert(0,'/Users/dl264294/Desktop/github/flowpm/')
#from flowpm.spectrum_for_cosmology import power_spectrum_for_cosmology
from flowpm.tfbackground import cosmo,z2a,a2z,rad_comoving_distance, a_of_chi
from flowpm.tfpower import linear_matter_power
from flowpm.angular_power_tf import measure_power_spectrum_tf
from flowpm.tfpower import linear_matter_power
from flowpm.raytracing import  lightcone, Born
import flowpm 


Populating the interactive namespace from numpy and matplotlib


In [2]:
nc=[32,32,320]   # size of the cube, number of cells
plane_size=32                    # number of pixel for x and  y 
Boxsize=[200,200,2000]          # Physical size of the cube
field=5.
n_steps=10

In [3]:
@tf.function
def power_spectrum_for_cosmology(
              Omega0_m,
              sigma8):
    cosmology=cosmo
    cosmology['Omega0_m']=tf.convert_to_tensor(Omega0_m,dtype=tf.float32)
    cosmology['sigma8']=tf.convert_to_tensor(sigma8,dtype=tf.float32)
    a_s=z2a(1.00)
    r = tf.linspace(0,2000,10)
    a=a_of_chi(cosmology,r)
    ds=rad_comoving_distance(cosmology,a_s)
    init_stages = tf.linspace(0.1, a[-1], 4)
    initial_conditions = flowpm.linear_field(nc,    
                                            Boxsize, 
                                             lambda k: tf.cast(linear_matter_power(cosmo, k), tf.complex64),         
                                             batch_size=1)
    # Sample particles
    state = flowpm.lpt_init(initial_conditions, 0.1)   
    # Evolve particles down to z=0
    final_state = flowpm.nbody(state, init_stages, nc)         
    # Retrieve final density field
    state, lps_a, lps=lightcone(final_state, a[::-1], 
                                  nc, 
                                    field*60/plane_size, plane_size,
              cosmology)
    k_map=Born(lps_a,lps,ds,nc,Boxsize,plane_size,field,cosmology)
    k_map=tf.cast(k_map,dtype=tf.complex64)
    ell, power_spectrum=measure_power_spectrum_tf(k_map,field,plane_size)
    return ell, power_spectrum, k_map



In [ ]:
ell3, power_spectrum3, kmap=power_spectrum_for_cosmology(0.3075,0.8159)

In [4]:
Omega0_m= tf.constant(0.3075)
sigma8=tf.constant(0.8159)
with tf.GradientTape() as tape:
    tape.watch(Omega0_m)
    ell, power_spectrum,kmap= power_spectrum_for_cosmology(
              Omega0_m,
              sigma8)
dy_dx = tape.gradient(power_spectrum, Omega0_m)
print(dy_dx)

Instructions for updating:
back_prop=False is deprecated. Consider using tf.stop_gradient instead.
Instead of:
results = tf.while_loop(c, b, vars, back_prop=False)
Use:
results = tf.nest.map_structure(tf.stop_gradient, tf.while_loop(c, b, vars))


TypeError: Expected int32, got 0.0 of type 'float' instead.